In [12]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keys import api, secret
from pybit.unified_trading import HTTP


session = HTTP(
    api_key=api,
    api_secret=secret)

session = HTTP(api_key=api, api_secret=secret)


In [13]:

def get_klines():
    try:
        resp = session.get_kline(
            category='linear',
            symbol='BTCUSDT',
            interval=1,
            limit=750
        )['result']['list']
        df = pd.DataFrame(resp)
        df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover']
        df = df.set_index('Time').astype(float).iloc[::-1]
        return df
    except Exception as e:
        print(f"Error fetching klines: {e}")
        return None

In [ ]:
df=get_klines()
ema_12 = df['Close'].ewm(span=12, min_periods=12).mean()
ema_26 = df['Close'].ewm(span=26, min_periods=26).mean()
macd_line = ema_12 - ema_26
signal_line = macd_line.ewm(span=9, min_periods=9).mean()
df['MACD'] = macd_line
df['Signal_Line'] = signal_line

# Calculate 200-period Moving Average
df['200_MA'] = df['Close'].rolling(window=200).mean()

In [ ]:
# Generate buy/sell signals
df['Buy_Signal'] = (df['MACD'] > df['Signal_Line']) & (df['Close'] > df['200_MA'])&(df['MACD']>5)
df['Sell_Signal'] = (df['MACD'] < df['Signal_Line']) & (df['Close'] < df['200_MA'])&(df['MACD']>-5)


In [ ]:
# Implement trading strategy
df['Position'] = np.where((df['MACD'] > df['Signal_Line']) & (df['Close'] > df['200_MA']), 1, 0)
df['Position'] = np.where((df['MACD'] < df['Signal_Line']) & (df['Close'] < df['200_MA']), -1, df['Position'])

# Calculate returns
df['Returns'] = df['Close'].pct_change() * df['Position'].shift(1)


In [ ]:

# Calculate returns
df['Returns'] = df['Close'].pct_change() * df['Position'].shift(1)

# Print cumulative returns
cumulative_returns = (df['Returns'] + 1).cumprod() - 1
print("Cumulative Returns:", cumulative_returns.iloc[-1])

In [ ]:
# Plot results
plt.figure(figsize=(14,7))
plt.plot(df['Close'], label='Price')
plt.plot(df['200_MA'], label='200 MA', color='orange')
plt.scatter(df[df['Position'] == 1].index, df[df['Position'] == 1]['Close'], label='Buy Signal', marker='^', color='g')
plt.scatter(df[df['Position'] == -1].index, df[df['Position'] == -1]['Close'], label='Sell Signal', marker='v', color='r')
plt.legend()
plt.title('MACD and 200 SMA Trading Strategy')
plt.show()

In [ ]:
def backtest_strategy(data):
    initial_capital = 10000 # Initial capital in USD
    capital = initial_capital
    position = 0  # 0: No position, 1: Long position, -1: Short position
    buy_price = 0
    sell_price = 0
    trades = []

    for i in range(1, len(data)):
        if data['Buy_Signal'][i] and position == 0:
            position = 1
            buy_price = data['Close'][i]
            capital -= buy_price
        elif data['Sell_Signal'][i] and position == 1:
            position = 0
            sell_price = data['Close'][i]
            capital += sell_price
            trades.append((buy_price, sell_price))

    # Calculate final capital and returns
    final_value = capital + (position * data['Close'].iloc[-1])
    returns = (final_value - initial_capital) / initial_capital * 100

    return final_value, returns, trades


In [ ]:
data=get_klines()
data['200_MA'] = data['Close'].rolling(window=200).mean()

# Calculate MACD and Signal line
data['MACD'], data['Signal'] = calculate_macd(data)

# Generate buy/sell signals with stop loss and take profit
data['Buy_Signal'], data['Sell_Signal'] = generate_signals(data)

# Perform backtesting
final_value, returns, trades = backtest_strategy(data)

print("Final Portfolio Value: $", final_value)
print("Total Return: ", returns, "%")

In [ ]:
# Plotting
plt.figure(figsize=(14,7))
plt.plot(data['Close'], label='Close Price', alpha=0.5)
plt.plot(data['200_MA'], label='200-day MA', alpha=0.5)
plt.scatter(data.index[data['Buy_Signal']], data['Close'][data['Buy_Signal']], marker='^', color='g', label='Buy Signal')
plt.scatter(data.index[data['Sell_Signal']], data['Close'][data['Sell_Signal']], marker='v', color='r', label='Sell Signal')
plt.title('Enhanced Crypto Trading Strategy with MACD, 200-day MA, Stop Loss, and Take Profit')
plt.legend()
plt.show()

In [ ]:
def get_tp_spl(expected_profit,amount,price,side):
    expected_profit =25
    expected_loss =expected_profit/3

   

    perct_increae= (expected_profit/amount)
    perct_decreae= (expected_loss/amount)

    ltp =price*(1+perct_increae)
    lsl =price*(1-perct_decreae)

    stp =price*(1-perct_increae)
    ssl =price*(1+perct_decreae)

    if side =='buy':
      return (ltp,lsl)
    
    else:
       return(stp,ssl)
    


    






In [3]:
from pybit.unified_trading import HTTP
session = HTTP(
    testnet=True,
    api_key="vHnoV1kQTgYsmhaY2s",
    api_secret="C5lVp013avzhEZgojyYxiT4TzwnZOX70gu8n",
)
print(session.place_order(
    category="spot",
    symbol="BTCUSDT",
    side="Buy",
    orderType="Limit",
    qty="0.1",
    price="15600",
    timeInForce="PostOnly",
    orderLinkId="spot-test-postonly",
    isLeverage=0,
    orderFilter="Order",
))

{'retCode': 0, 'retMsg': 'OK', 'result': {'orderId': '1630539316747043328', 'orderLinkId': 'spot-test-postonly'}, 'retExtInfo': {}, 'time': 1709111433534}


In [11]:
open=[1,2,3,5]

if len(open)==5:
    print(len(open))

In [32]:
from ta import momentum
def calculate_ema(df):
    close_df = df["Close"]
    ema9 = close_df.ewm(span=12, adjust=False).mean()
    ema14 = close_df.ewm(span=14, adjust=False).mean()
    ema21 = close_df.ewm(span=21, adjust=False).mean()
    ema26 = close_df.ewm(span=26, adjust=False).mean()
    ema50 = close_df.ewm(span=50, adjust=False).mean()
    ema200 = close_df.ewm(span=200, adjust=False).mean()

    return ema9.iloc[-1],ema14.iloc[-1],ema21.iloc[-1], ema26.iloc[-1],ema50.iloc[-1],ema200.iloc[-1],close_df.iloc[-1]



def three_moving_average_rsi_strategy(df):
    _,_,ema21,_,ema50,ema200,last_price=calculate_ema(df=df)

    rsi =momentum.rsi(close=df['Close'],window=14)
    print(rsi.iloc[-2])

    if( last_price >ema21) and (ema21>ema50) and (ema50>ema200) and (rsi <50):
        print('Buy')


In [33]:
df =get_klines()

three_moving_average_rsi_strategy(df)

63.66355175639627


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().